In [2]:
import pandas as pd
df=pd.read_csv('diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.shape

(768, 9)

In [7]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [21]:
x=df.drop("Outcome",axis=1)
x
y=df['Outcome']
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [22]:
from sklearn.model_selection import train_test_split
# implementing train test split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=2)

In [24]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(x_train,y_train)
rfc_predict=rfc.predict(x_test)

C:\Users\apssdc\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [25]:
# Evaluating performance
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix


In [27]:
rfc_cv_score=cross_val_score(rfc,x,y,cv=10,scoring='roc_auc')

In [33]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test,rfc_predict))
print("\n")

=== Confusion Matrix ===
[[149  22]
 [ 41  42]]




In [30]:
print("@@@ clasification report@@@")
print(classification_report(y_test,rfc_predict))
print("\n")

@@@ clasification report@@@
              precision    recall  f1-score   support

           0       0.78      0.87      0.83       171
           1       0.66      0.51      0.57        83

   micro avg       0.75      0.75      0.75       254
   macro avg       0.72      0.69      0.70       254
weighted avg       0.74      0.75      0.74       254





In [31]:
print("!!! all AUC scores !!!")
print(rfc_cv_score)

!!! all AUC scores !!!
[0.76814815 0.81481481 0.7862963  0.64296296 0.78481481 0.89444444
 0.84555556 0.85333333 0.75615385 0.83038462]


In [32]:
print("%%% Mean of AUC score %%%")
print("mean of auc score",rfc_cv_score.mean())

%%% Mean of AUC score %%%
mean of auc score 0.7976908831908832


In [39]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=11)]
max_features=['auto','sqrt']
# max depth
max_depth=[int(x) for x in np.linspace(100,500,num=11)]
max_depth.append(None)
# creat random grid
random_grid={'n_estimators':n_estimators,'max_features':max_features,'max_depth':max_depth}
rfc_random=RandomizedSearchCV(estimator=rfc,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,random_state=2,n_jobs=-1)
rfc_random.fit(x_train,y_train)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.6min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 380, 560, 740, 920, 1100, 1280, 1460, 1640, 1820, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [100, 140, 180, 220, 260, 300, 340, 380, 420, 460, 500, None]},
          pre_dispatch='2*n_jobs', random_state=2, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [40]:
print(rfc_random.best_params_)

{'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 260}


In [42]:
rfc=RandomForestClassifier(n_estimators=200,max_depth=260,max_features='sqrt',)
rfc.fit(x_train,y_train)
rfc_predict=rfc.predict(x_test)
rfc_cv_score=cross_val_score(rfc,x,y,cv=10,scoring='roc_auc')
print("### confusion matrix ###")
print(confusion_matrix(y_test,rfc_predict))

### confusion matrix ###
[[150  21]
 [ 38  45]]


In [43]:
print("### clasification repport ###")
print(classification_report(y_test,rfc_predict))

### clasification repport ###
              precision    recall  f1-score   support

           0       0.80      0.88      0.84       171
           1       0.68      0.54      0.60        83

   micro avg       0.77      0.77      0.77       254
   macro avg       0.74      0.71      0.72       254
weighted avg       0.76      0.77      0.76       254



In [44]:
print("### all aus sore ###")
print(rfc_cv_score)

### all aus sore ###
[0.78740741 0.82703704 0.8462963  0.71888889 0.80518519 0.85518519
 0.85925926 0.91185185 0.82       0.85730769]


In [45]:
print("### mean of all aus score ###")
print(rfc_cv_score.mean())

### mean of all aus score ###
0.8288418803418803


In [47]:
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [52]:

A=rfc.predict([[1,89,66,23,94,28.1,0.167,21]])
A

array([0], dtype=int64)